# Reinforcement Learning

This notebook serves as the supporting material for the chapter **Reinforcement Learning**. Here we'll examine how an agent can learn what to do in the absence of labeled examples of what to do, from rewards and punishments. This notebook illustrates the use of the [reinforcement](https://github.com/aimacode/aima-java/tree/AIMA3e/aima-core/src/main/java/aima/core/learning/reinforcement) package of the code repository. So let's begin with "What is reinforcement?". 

Consider an example of a problem of learning chess. A supervised agent needs to be told the correct move for each position it encounters, but such feedback is seldom available. Therefore, in the absence of feedback, the agent needs to know, that something good has happened when it accidentally checkmates its opponent and that something bad has happened when it gets checkmated. This kind of feedback is called a **reward** or **reinforcement**. Reinforcement learning differs from the supervised learning in a way that in supervised learning the training data has the answer label with it so the model is trained with the correct answer itself whereas in reinforcement learning, there is no answer but the reinforcement agent itself decides what to do to perform the given task. In the absence of the training dataset, it is bound to learn from its experience. 

Usually, in game playing, it is very hard for a human to provide accurate and consistent evaluations of a large number of positions. Therefore, the program is told when it has won or lost, and the agent uses this information to learn a reasonably accurate evaluation function.

In [1]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar